In [1]:
# -*- coding: utf-8 -*-
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
import sys
from PIL import Image, ImageTk, ImageDraw, ImageFilter
try:
    import Tkinter as tk
except ImportError:
    import tkinter as tk  # for Python 3
    
import tkinter.filedialog
import os

#from gahuu import convert_img, digit
import gahuu

import cv2
import imageio

##### 以下はGUIアプリケーション用の内容

class Application(tk.Frame):
    # 初期化用関数
    def __init__(self, master=None):
        tk.Frame.__init__(self, master)
        self.pack()
        self.w = 500 # 一つの描画領域の幅
        self.h = 500 # 一つの描画領域の高さ
        self.create_widgets()

    # 様々なGUI部品を構築
    def create_widgets(self):
        w = self.w
        h = self.h
        
        # 描画するための領域
        self.canvas = tk.Canvas(self, width=w, height=h, bg='white')
        # self.canvasを左端に配置
        self.canvas.grid(row=0, column=0)


        # スタイル用に入力された画像を表示するキャンバス
        self.recog_canvas = tk.Canvas(self, width=w, height=h, bg='white')
        # self.canvasを中央に配置
        self.recog_canvas.grid(row=0, column=1)

        # 結果表示用のキャンバス
        self.digit_canvas = tk.Canvas(self, width=w, height=h, bg='white')
        # self.digit_canvasを右端に配置
        self.digit_canvas.grid(row=0, column=2)
        
        # 入力画像表示ボタン
        frame_in_btn = tk.Frame(self)
        frame_in_btn.grid(row=1, column=0)
        self.in_btn = tk.Button(frame_in_btn, text='オリジナル入力', command=self.in_recog).pack(side = 'left')
        self.clear_in_btn = tk.Button(frame_in_btn, text='クリア', command=self.in_clear).pack(side = 'left')
        
        # スタイル用に入力された画像表示ボタン
        frame_btn = tk.Frame(self)
        frame_btn.grid(row=1, column=1)
        self.recog_btn = tk.Button(frame_btn, text='スタイル入力', command=self.recog).pack(side = 'left')
        self.clear_btn = tk.Button(frame_btn, text='クリア', command=self.clear).pack(side = 'left')
        
        #画風変換結果表示ボタン
        frame_digit_btn = tk.Frame(self)
        frame_digit_btn.grid(row=1, column=2)
        self.digit_btn = tk.Button(frame_digit_btn, text='画風変換', command=self.digit_output).pack(side = 'left')
        self.clear_btn = tk.Button(frame_digit_btn, text='クリア', command=self.out_clear).pack(side = 'left')
              
    # 入力画像表示ボタンが押されたときに実行される関数
    def in_recog(self):
        filepath = tkinter.filedialog.askopenfilename(initialdir='input_original/')
        dirname, basename = os.path.split(filepath)
        in_name = os.path.join(os.path.basename(dirname),basename)
        
        #digit_outputで学習させるためにself付きの変数に格納
        self.content_image = gahuu.convert_img(in_name)
        
        #結果画像表示の為に保存名取得
        self.basename_without_ext = os.path.splitext(os.path.basename(filepath))[0]
        
        self.in_img = Image.open(in_name)
        w = self.in_img.width # 横幅を取得                                            
        h = self.in_img.height # 縦幅を取得
        
        if w > h:
            self.in_img = self.in_img.resize(( int(w * (500/w)), int(h * (500/w)) ))
            self.in_img = ImageTk.PhotoImage(self.in_img)
            self.canvas.create_image(0, 250-(h*(500/w)/2), image=self.in_img, anchor=tk.NW)
        else:
            self.in_img = self.in_img.resize(( int(w * (500/h)), int(h * (500/h)) ))
            self.in_img = ImageTk.PhotoImage(self.in_img)
            self.canvas.create_image(250-(w*(500/h)/2), 0, image=self.in_img, anchor=tk.NW)
            
    # スタイル用画像表示ボタンが押されたときに実行される関数
    def recog(self):
        filepath = tkinter.filedialog.askopenfilename(initialdir='input_style/')
        dirname, basename = os.path.split(filepath)
        style_name = os.path.join(os.path.basename(dirname),basename)
        
        #digit_outputで学習させるためにself付きの変数に格納
        self.style_image = gahuu.convert_img(style_name)
        
        self.img = Image.open(style_name)
        w = self.img.width # 横幅を取得                                            
        h = self.img.height # 縦幅を取得
        
        if w > h:
            self.img = self.img.resize(( int(w * (500/w)), int(h * (500/w)) ))
            self.img = ImageTk.PhotoImage(self.img)
            self.recog_canvas.create_image(0, 250-(h*(500/w)/2), image=self.img, anchor=tk.NW)
        else:
            self.img = self.img.resize(( int(w * (500/h)), int(h * (500/h)) ))
            self.img = ImageTk.PhotoImage(self.img)
            self.recog_canvas.create_image(250-(w*(500/h)/2), 0, image=self.img, anchor=tk.NW)
            
    def digit_output(self):
        #モジュール先の機械学習modelで変換
        stylized_image = gahuu.digit(self.content_image, self.style_image)
        
        #uint8形式の数値に変換（255かける）
        stylized_image_unit = tf.cast(stylized_image*255.0, tf.uint8)
        
        #jpegで書き出す為に3次元化（4次元テンソルだとエラーになった）
        stylized_image_unit_3d = tf.reshape(stylized_image_unit,
                                            [len(stylized_image_unit[0]), len(stylized_image_unit[0][1]), 3])
        
        #スタイル画像名に_convをつけてconvフォルダに画風変換のjpeg画像を書き出し
        imageio.imwrite("output_conv/{}_conv.jpg".format(self.basename_without_ext),stylized_image_unit_3d)
        
        #jpeg画像をPIL化しTKinterのcanvasに表示
        stylized_img = Image.open("output_conv/{}_conv.jpg".format(self.basename_without_ext))
        
        w = stylized_img.width # 横幅を取得                                            
        h = stylized_img.height # 縦幅を取得        
        if w > h:
            stylized_img = stylized_img.resize(( int(w * (500/w)), int(h * (500/w)) ))
            self.stylized_img = ImageTk.PhotoImage(stylized_img)
            self.digit_canvas.create_image(0, 250-(h*(500/w)/2), image=self.stylized_img, anchor=tk.NW)
        else:
            stylized_img = stylized_img.resize(( int(w * (500/h)), int(h * (500/h)) ))
            self.stylized_img = ImageTk.PhotoImage(stylized_img)
            self.digit_canvas.create_image(250-(w*(500/h)/2), 0, image=self.stylized_img, anchor=tk.NW)
    
    # クリアボタンが押されたときに実行される関数
    def in_clear(self):
        # 左の描画領域をクリア
        self.canvas.delete('all')
        
    def clear(self):
        # 真ん中の描画領域をクリア
        self.recog_canvas.delete('all')
        
    def out_clear(self):
        # 右の描画領域をクリア
        self.digit_canvas.delete('all')

root = tk.Tk()
app = Application(master=root)
app.master.title('高速画風変換')
app.mainloop()

In [ ]:
self.content_image.shape